In [17]:
!pip install requests pandas tqdm


***Uploading Dataset***

In [19]:
from google.colab import files
uploaded = files.upload()


Saving Wallet id - Sheet1.csv to Wallet id - Sheet1.csv


***Loading Columns***

In [20]:
import pandas as pd

wallets_df = pd.read_csv("Wallet id - Sheet1.csv")
wallets_df.columns = [col.strip().lower().replace(" ", "_") for col in wallets_df.columns]
wallets = wallets_df['wallet_id'].dropna().str.lower().unique().tolist()
wallets[:5]


['0x0039f22efb07a647557c7c5d17854cfd6d489ef3',
 '0x06b51c6882b27cb05e712185531c1f74996dd988',
 '0x0795732aacc448030ef374374eaae57d2965c16c',
 '0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9',
 '0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae']

***Trying CompoundV2***

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

# Replace with your list of wallet addresses
wallets = [
    '0xfaa0768bde629806739c3a4620656c5d26f44ef2',
    # Add more addresses here
]

# Compound V2 Subgraph API
COMPOUND_V2_SUBGRAPH = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

# GraphQL Query Template
def make_query(wallet):
    return {
        "query": """
        {
          account(id: "%s") {
            id
            tokens {
              symbol
              supplyBalanceUnderlying
              borrowBalanceUnderlying
              lifetimeBorrowInterestAccrued
              lifetimeSupplyInterestAccrued
              enteredMarket
            }
          }
        }
        """ % wallet.lower()
    }

# Fetch data
def fetch_wallet_data(wallets):
    records = []
    for wallet in tqdm(wallets):
        res = requests.post(COMPOUND_V2_SUBGRAPH, json=make_query(wallet))
        data = res.json()
        tokens = data.get("data", {}).get("account", {}).get("tokens", [])
        for t in tokens:
            records.append({
                "wallet": wallet,
                "symbol": t['symbol'],
                "supply": float(t['supplyBalanceUnderlying'] or 0),
                "borrow": float(t['borrowBalanceUnderlying'] or 0),
                "interest_borrowed": float(t['lifetimeBorrowInterestAccrued'] or 0),
                "interest_supplied": float(t['lifetimeSupplyInterestAccrued'] or 0),
                "entered_market": t['enteredMarket']
            })
    return pd.DataFrame(records)

df_raw = fetch_wallet_data(wallets)
df_raw.head()


***Trying AaveV2***

In [21]:
import requests
from tqdm import tqdm

AAVE_V2_SUBGRAPH = "https://api.thegraph.com/subgraphs/name/aave/protocol-v2"

def build_aave_query(wallet):
    return {
        "query": f"""
        {{
          userReserves(where: {{ user: "{wallet}" }}) {{
            user
            reserve {{
              symbol
            }}
            currentTotalDebt
            currentATokenBalance
            usageAsCollateralEnabledOnUser
          }}
        }}
        """
    }

def fetch_aave_data(wallets):
    records = []
    for wallet in tqdm(wallets):
        try:
            res = requests.post(AAVE_V2_SUBGRAPH, json=build_aave_query(wallet))
            data = res.json()
            for entry in data['data']['userReserves']:
                records.append({
                    "wallet": wallet,
                    "symbol": entry["reserve"]["symbol"],
                    "debt": float(entry["currentTotalDebt"]),
                    "supply": float(entry["currentATokenBalance"]),
                    "collateral_enabled": entry["usageAsCollateralEnabledOnUser"]
                })
        except Exception as e:
            print(f"Failed for wallet {wallet}: {e}")
    return pd.DataFrame(records)

aave_df = fetch_aave_data(wallets)
aave_df.to_csv("aave_user_data.csv", index=False)


  2%|▏         | 2/103 [00:00<00:12,  8.17it/s]

Failed for wallet 0x0039f22efb07a647557c7c5d17854cfd6d489ef3: 'data'
Failed for wallet 0x06b51c6882b27cb05e712185531c1f74996dd988: 'data'


  4%|▍         | 4/103 [00:00<00:11,  8.62it/s]

Failed for wallet 0x0795732aacc448030ef374374eaae57d2965c16c: 'data'
Failed for wallet 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9: 'data'


  6%|▌         | 6/103 [00:00<00:10,  8.91it/s]

...

Failed for wallet 0xf60304b534f74977e159b2e159e135475c245526: 'data'
Failed for wallet 0xf67e8e5805835465f7eba988259db882ab726800: 'data'


 99%|█████████▉| 102/103 [00:11<00:00,  9.19it/s]

Failed for wallet 0xf7aa5d0752cfcd41b0a5945867d619a80c405e52: 'data'
Failed for wallet 0xf80a8b9cfff0febf49914c269fb8aead4a22f847: 'data'


100%|██████████| 103/103 [00:11<00:00,  9.14it/s]

Failed for wallet 0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac: 'data'


In [22]:
aave_df = fetch_aave_data(wallets)
aave_df.to_csv("aave_user_data.csv", index=False)


  2%|▏         | 2/103 [00:00<00:12,  8.22it/s]

Failed for wallet 0x0039f22efb07a647557c7c5d17854cfd6d489ef3: 'data'
Failed for wallet 0x06b51c6882b27cb05e712185531c1f74996dd988: 'data'


  4%|▍         | 4/103 [00:00<00:10,  9.12it/s]

Failed for wallet 0x0795732aacc448030ef374374eaae57d2965c16c: 'data'
Failed for wallet 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9: 'data'


  6%|▌         | 6/103 [00:00<00:10,  8.94it/s]

...

Failed for wallet 0xf60304b534f74977e159b2e159e135475c245526: 'data'
Failed for wallet 0xf67e8e5805835465f7eba988259db882ab726800: 'data'


 99%|█████████▉| 102/103 [00:11<00:00,  9.40it/s]

Failed for wallet 0xf7aa5d0752cfcd41b0a5945867d619a80c405e52: 'data'
Failed for wallet 0xf80a8b9cfff0febf49914c269fb8aead4a22f847: 'data'


100%|██████████| 103/103 [00:11<00:00,  9.23it/s]

Failed for wallet 0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac: 'data'


***Trying AaveV2 Again***

In [25]:
import pandas as pd
import requests
import time

# Load wallet addresses from uploaded CSV
wallet_df = pd.read_csv("/content/Wallet id - Sheet1.csv")  # adjust path if needed
wallets = wallet_df['wallet_id'].dropna().str.lower().unique().tolist()

# Aave V2 subgraph endpoint
AAVE_V2_URL = "https://api.thegraph.com/subgraphs/name/aave/protocol-v2"

# GraphQL query for Aave V2 protocol data
def fetch_aave_data(wallet):
    query = """
    {
      user(id: "%s") {
        id
        reserves {
          reserve {
            symbol
          }
          currentATokenBalance
          currentTotalDebt
          usageAsCollateralEnabledOnUser
        }
      }
    }
    """ % wallet.lower()

    response = requests.post(AAVE_V2_URL, json={"query": query})
    if response.ok:
        return response.json().get("data", {}).get("user")
    else:
        print(f"Failed for wallet: {wallet}")
        return None

# Fetch data for all wallets
aave_data = []
for i, wallet in enumerate(wallets):
    user_data = fetch_aave_data(wallet)
    if user_data:
        for reserve in user_data['reserves']:
            aave_data.append({
                "wallet": wallet,
                "symbol": reserve["reserve"]["symbol"],
                "aToken_balance": float(reserve["currentATokenBalance"]),
                "debt": float(reserve["currentTotalDebt"]),
                "collateral_enabled": reserve["usageAsCollateralEnabledOnUser"]
            })
    time.sleep(0.3)  # avoid rate limits

# Create DataFrame
df = pd.DataFrame(aave_data)
df.head()


""


***Trying AaveV3***

In [26]:
import pandas as pd
import requests
import time

# Load wallet CSV again (uploaded previously)
wallet_df = pd.read_csv("/content/Wallet id - Sheet1.csv")
wallets = wallet_df['wallet_id'].dropna().str.lower().unique().tolist()

# Aave V3 Subgraph endpoint
AAVE_V3_URL = "https://api.thegraph.com/subgraphs/name/aave/protocol-v3"

# GraphQL query function
def fetch_aave_v3_data(wallet):
    query = """
    {
      user(id: "%s") {
        id
        reserves {
          reserve {
            symbol
          }
          scaledATokenBalance
          currentTotalDebt
          usageAsCollateralEnabledOnUser
        }
      }
    }
    """ % wallet.lower()

    response = requests.post(AAVE_V3_URL, json={"query": query})
    if response.ok:
        return response.json().get("data", {}).get("user")
    else:
        print(f"Failed for {wallet}")
        return None

# Loop through all wallets
aave_v3_data = []
for i, wallet in enumerate(wallets):
    print(f"Fetching {i+1}/{len(wallets)}: {wallet}")
    user_data = fetch_aave_v3_data(wallet)
    if user_data:
        for reserve in user_data['reserves']:
            aave_v3_data.append({
                "wallet": wallet,
                "symbol": reserve["reserve"]["symbol"],
                "aToken_balance": float(reserve["scaledATokenBalance"]),
                "debt": float(reserve["currentTotalDebt"]),
                "collateral_enabled": reserve["usageAsCollateralEnabledOnUser"]
            })
    time.sleep(0.3)  # rate limiting

# Convert to DataFrame
df = pd.DataFrame(aave_v3_data)
df.head()


Fetching 1/103: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Fetching 2/103: 0x06b51c6882b27cb05e712185531c1f74996dd988
Fetching 3/103: 0x0795732aacc448030ef374374eaae57d2965c16c
Fetching 4/103: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Fetching 5/103: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Fetching 6/103: 0x104ae61d8d487ad689969a17807ddc338b445416
Fetching 7/103: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Fetching 8/103: 0x124853fecb522c57d9bd5c21231058696ca6d596
Fetching 9/103: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Fetching 10/103: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Fetching 11/103: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
Fetching 12/103: 0x19df3e87f73c4aaf4809295561465b993e102668
Fetching 13/103: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
Fetching 14/103: 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f
Fetching 15/103: 0x1e43dacdcf863676a6bec8f7d6896d6252fac669
Fetching 16/103: 0x22d7510588d90ed5a87e0f838391aaafa707c34b
Fetching 17/103: 0x24b3460622d835c56d9a4fe352966b

""


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

***Trying CompoundV3***

In [27]:
import pandas as pd
import requests
import time

# Load your wallet list
wallet_df = pd.read_csv("/content/Wallet id - Sheet1.csv")
wallets = wallet_df['wallet_id'].dropna().str.lower().unique().tolist()

# Compound V3 (community-maintained) subgraph endpoint
COMPOUND_V3_URL = "https://api.thegraph.com/subgraphs/name/0xngmi/compound-v3"

# GraphQL query template
def fetch_compound_v3(wallet):
    query = """
    {
      user(id: "%s") {
        id
        collateralBalance
        baseBalance
        debtValue
        baseToken {
          symbol
        }
        positions {
          asset {
            symbol
          }
          balance
          isCollateral
        }
      }
    }
    """ % wallet.lower()

    response = requests.post(COMPOUND_V3_URL, json={"query": query})
    if response.ok:
        return response.json().get("data", {}).get("user")
    else:
        print(f"Error for {wallet}")
        return None

# Fetch data for all wallets
compound_v3_data = []

for i, wallet in enumerate(wallets):
    print(f"{i+1}/{len(wallets)} — Fetching: {wallet}")
    user = fetch_compound_v3(wallet)
    if user:
        for pos in user.get("positions", []):
            compound_v3_data.append({
                "wallet": wallet,
                "base_token": user.get("baseToken", {}).get("symbol"),
                "collateral_balance": float(user.get("collateralBalance", 0)),
                "base_balance": float(user.get("baseBalance", 0)),
                "debt_value": float(user.get("debtValue", 0)),
                "asset_symbol": pos["asset"]["symbol"],
                "position_balance": float(pos["balance"]),
                "is_collateral": pos["isCollateral"]
            })
    time.sleep(0.3)  # To avoid rate limits

# Create DataFrame
compound_v3_df = pd.DataFrame(compound_v3_data)
compound_v3_df.head()


1/103 — Fetching: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
2/103 — Fetching: 0x06b51c6882b27cb05e712185531c1f74996dd988
3/103 — Fetching: 0x0795732aacc448030ef374374eaae57d2965c16c
4/103 — Fetching: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
5/103 — Fetching: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
6/103 — Fetching: 0x104ae61d8d487ad689969a17807ddc338b445416
7/103 — Fetching: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
8/103 — Fetching: 0x124853fecb522c57d9bd5c21231058696ca6d596
9/103 — Fetching: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
10/103 — Fetching: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
11/103 — Fetching: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
12/103 — Fetching: 0x19df3e87f73c4aaf4809295561465b993e102668
13/103 — Fetching: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
14/103 — Fetching: 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f
15/103 — Fetching: 0x1e43dacdcf863676a6bec8f7d6896d6252fac669
16/103 — Fetching: 0x22d7510588d90ed5a87e0f838391aaafa707c34b
17/103 — Fetching

""


## ***Colab Code to Simulate Data***

In [32]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime, timedelta

# Step 1: Generate 100 mock wallet addresses
def generate_wallet_address():
    return "0x" + ''.join(random.choices('abcdef' + string.digits, k=40))

wallets = [generate_wallet_address() for _ in range(100)]

# Step 2: Define mock Compound transaction actions
compound_actions = ['mint', 'borrow', 'repayBorrow', 'redeem', 'liquidateBorrow']

# Step 3: Generate random transaction history per wallet
mock_data = []

for wallet in wallets:
    num_tx = random.randint(10, 50)
    start_date = datetime(2023, 1, 1)
    for _ in range(num_tx):
        action = random.choice(compound_actions)
        amount = round(np.random.exponential(scale=1000), 2)  # simulate skewed DeFi values
        timestamp = start_date + timedelta(days=random.randint(0, 500))
        mock_data.append({
            "wallet_id": wallet,
            "action": action,
            "amount": amount,
            "timestamp": timestamp
        })

# Step 4: Create DataFrame
df = pd.DataFrame(mock_data)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Preview
df.head()


,wallet_id,action,amount,timestamp
0,0x7d96ca126d0697d55c11f6062736f46f9a986535,liquidateBorrow,105.26,2024-03-22
1,0x7d96ca126d0697d55c11f6062736f46f9a986535,borrow,685.72,2023-06-03
2,0x7d96ca126d0697d55c11f6062736f46f9a986535,mint,845.24,2023-01-31
3,0x7d96ca126d0697d55c11f6062736f46f9a986535,borrow,333.21,2024-01-16
4,0x7d96ca126d0697d55c11f6062736f46f9a986535,repayBorrow,2628.74,2023-03-22


***Feature Engineering + Risk Scoring Code***

In [34]:
# Step 5: Feature Engineering
agg_features = df.pivot_table(
    index='wallet_id',
    columns='action',
    values='amount',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Add derived features
agg_features['total_txn'] = df.groupby('wallet_id').size().values
agg_features['net_balance'] = agg_features.get('mint', 0) - agg_features.get('redeem', 0)
agg_features['repay_ratio'] = agg_features.get('repayBorrow', 0) / (agg_features.get('borrow', 1))
agg_features['liquidation_risk'] = agg_features.get('liquidateBorrow', 0)

# Step 6: Scoring Logic
from sklearn.preprocessing import MinMaxScaler

# Select features to score
features_to_score = ['mint', 'borrow', 'repayBorrow', 'redeem', 'net_balance', 'repay_ratio', 'liquidation_risk', 'total_txn']
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(agg_features[features_to_score])

# Simple scoring: higher mint/repay/txn count, lower liquidation = good
weights = np.array([1.2, 1.0, 1.5, 1.0, 1.3, 1.5, -2.0, 1.2])  # tuned weights
agg_features['score_raw'] = scaled_features.dot(weights)

# Normalize to 0–1000
agg_features['score'] = MinMaxScaler(feature_range=(0, 1000)).fit_transform(agg_features[['score_raw']]).round().astype(int)

# Final output
final_scores = agg_features[['wallet_id', 'score']]
final_scores.head()


action,wallet_id,score
0,0x02aba2d7e12c8eb700d8ff34fce856ce84729fce,469
1,0x057b8c8024114bdd548d2307ea05358712c54798,1000
2,0x09931927b32a71660cb84495284c242b145db893,94
3,0x0b941f9cd795bbbd3f1158e3bad540098e2ed9a4,173
4,0x0f3555bffb2bb7e735250b1177daef0590348041,393


***Export Final CSV***

In [35]:
# Export to CSV
final_scores.to_csv("wallet_risk_scores.csv", index=False)
